## 构建KD树

如何搜索Kd-tree找到目标点P最邻近的k个点？

1. 根据P的坐标值和每个节点的切分向下搜索（也就是比较该结点切分维度上的坐标值，比如一个结点是按照y维分的，就比较P点和结点的y坐标，若P点小，则走向左枝，若P点大，则走向右枝。

2. 当到达一个底部结点时，将其标记为已访问。

如果L中不足k个点，则将该点加入到L中；如果L不为空且该点与P的距离小于L中离P远的点的距离，则用该结点替换那个最远点 。

3. 当前结点不是顶端结点时或是顶端结点却未被标记已访问（若是且已被被标记访问则停止），则向上爬，若已被标记访问，则继续向上爬；若未被标记，则标记已访问，并依次执行下面两步：

1）L中不足k个点，则加入；若L已满，则计算P与该点距离，若小于L中最大距离，则替换之。

2）计算P和当前结点切分线的距离D，若D大于L中最大距离且L已满，则该结点的另一分支不会有更近的点，至此结束。若D小于L中最大距离或L未满，则该结点的另一分支可能有更近的点，继续在另一分支搜索（从1开始）。

In [9]:
import copy
import numpy as np
import pandas as pd
from queue import PriorityQueue
import queue

from sklearn import datasets

class Node:
    def __init__(self,i,v=None,l=None,r=None):
        self.index = i
        self.value = v
        self.left = l
        self.right = r

        
class KNN:
    
    queue = None
    tree = None
    k = 1
            
    def __init__(self,X_train=None,y_train=None):
        if not isinstance(X_train,pd.DataFrame):
            X_train = pd.DataFrame(X_train)
        if not isinstance(y_train,pd.Series):
            y_train = pd.Series(y_train)
        self.X_train = X_train
        self.y_train = y_train
        self.tree = self._build()
        
    def kneighbors(self,test_x,k=1):
        result = []
        for i in test_x:
            res = []
            self._search_K(i,k)
            while not self.queue.empty():
                row = self.queue.get()
                res.append(int(row[1][-1]))
            result.append(res)
        return result
    
    def predict(self,test_x,k=1):
        result = []
        for i in test_x:
            res = []
            self._search_K(i,k)
            while not self.queue.empty():
                row = self.queue.get()
                res.append(int(row[1][-1]))
            result.append(self._vote_(res))
        return result
    
    def _vote_(self,index_list):
        return self.y_train[np.argmax(np.bincount(index_list))]
    
    
    def _build(self):
        if self.X_train is not None and self.y_train is not None:
            # 将索引值加入最后一列
            combine = self.X_train.copy()
            combine['idx'] = pd.Series(self.X_train.index)
            print(combine.head())
            return self._build_tree(combine)
            print(self.tree.value)
        return None
    
    def _build_tree(self,dataset):
        if dataset.shape[0] == 0:
            return None
        i,left,mid,right = self._process_data(dataset)
        node = Node(i,mid.values[0],self._build_tree(left),self._build_tree(right))
        return node
    
    def _process_data(self,dataset):
        if dataset.shape[0] == 1:
            return 0,pd.DataFrame([]),dataset,pd.DataFrame([])
        #首先选择维度并根据该维度排序
        index = self._find_max_var(dataset)
        dataset.sort_values(index,inplace=True)
        # 选择中位数
        m,n = dataset.shape
        mid_index = int(np.floor((m-1)/2))
        left = dataset[0:mid_index].reset_index(drop=True)
        right = dataset[mid_index+1:].reset_index(drop=True)
        mid = dataset[mid_index:mid_index+1]
        return index,left,mid,right
    
    def _find_max_var(self,dataset):
        var = dataset.drop(['idx'],axis=1).var()
        return var.idxmax()
    
    # 欧拉距离
    def _f2_dis(self,node,search_node):
        return np.linalg.norm(node-search_node)
    
    def _search_K(self,x,k=1):
        if self.tree is not None:
            self.k = k
            self.queue = PriorityQueue(k)
            self._search_K_NN(x,self.tree)
    
    def _search_K_NN(self,x,node):
        # 直接访问到端节点
        index = node.index
        value = node.value
        other = None
        if x[index]<=value[index]:
            other = node.right
            if not node.left is None:
                self._search_K_NN(x,node.left)
        else:
            other = node.left
            if not node.right is None:
                self._search_K_NN(x,node.right)
        max_length = self._insert_(value,x)
        d_dis = self._f2_dis(np.array([x[index]]),np.array([value[index]]))
        if d_dis<=max_length:
            if not other is None:
                self._search_K_NN(x,other)
                
    def _insert_(self,node,search_node):
        length = self._f2_dis(node[:-1],search_node) 
        if self.queue.full():
            first = self.queue.get()
            if length >= -first[0]:
                node = first[1]
                length = -first[0]
        if not isinstance(node, list):
            node = node.tolist()
        self.queue.put((-length,node))
        first = self.queue.get()
        self.queue.put(first)
        return -first[0]

In [10]:
iris = datasets.load_iris()
x = iris['data']
y = iris['target']

In [11]:
k = KNN(X_train=x,y_train=y)

     0    1    2    3  idx
0  5.1  3.5  1.4  0.2    0
1  4.9  3.0  1.4  0.2    1
2  4.7  3.2  1.3  0.2    2
3  4.6  3.1  1.5  0.2    3
4  5.0  3.6  1.4  0.2    4


In [13]:
a = k.predict([[4.6,3,1.2,0.15],[4.6,3,1.2,0.15],[4.6,3,1.2,0.15]],3)
b = k.kneighbors([[4.6,3,1.2,0.15],[4.6,3,1.2,0.15],[4.6,3,1.2,0.15]],3)

In [14]:
print(a)
print(b)

[0, 0, 0]
[[12, 2, 38], [12, 2, 38], [12, 2, 38]]
